# Основы Python. Часть 4

## Метапрограммирование.

### Создание функций

In [1]:
def f(a, b):
    return a + b

In [2]:
s = f

s(1, 2)

3

In [3]:
def create_f():
    def f_template(a, b):
        return a + b
    return f

In [4]:
f1 = create_f()

f2 = create_f()

f1(1, 2)

3

In [5]:
%%time

from functools import wraps
import random
import time

def wait_random(min_wait=5, max_wait=10):
    def inner_function(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            time.sleep(random.randint(min_wait, max_wait))
            return func(*args, **kwargs)
        return wrapper
    return inner_function

@wait_random(4, 5)
def print_hello():
    print('Hello')
    
print_hello()

Hello
CPU times: user 2.94 ms, sys: 314 µs, total: 3.26 ms
Wall time: 5 s


### Простое создание методов класса

In [6]:
class A:
    name = 'is A'

def f(obj):
    print('f run: {}'.format(obj.name))

A.f = f

print(A.__dict__)

a = A()

a.f()

{'__module__': '__main__', 'name': 'is A', '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None, 'f': <function f at 0x7f142814f730>}
f run: is A


### `hasattr` `getattr` и `setattr`

In [7]:
class A:
    name = 'is A'
    
    @staticmethod
    def s():
        print('static member call')
    
    def f(self):
        print('object member call')

In [8]:
hasattr(A, 'name')

True

In [9]:
hasattr(A, 'f')

True

Можно получить доступ к атрибуту по его имени

In [10]:
_name = getattr(A, 'name')
_name

'is A'

In [11]:
_s = getattr(A, 's')
_s()

static member call


Ошибка, потому что методу f нужен один позиционный аргумент. В случае класса - это сам объект.

In [12]:
_f = getattr(A, 'f')
_f()

TypeError: f() missing 1 required positional argument: 'self'

Создадим объект и теперь можно его вызвать

In [13]:
a = A()

_f(a)

object member call


Можно прикрепить новую переменную к классу

In [14]:
setattr(A, 'value', 1)

A.value

1

Можно прикрепить новый метод к классу

In [16]:
def get_value(self):
    print('Object: {}'.format(self.name))

setattr(A, 'get_value', get_value)

Ошибка, потому что функции нужен аргумент.

In [17]:
A.get_value()

TypeError: get_value() missing 1 required positional argument: 'self'

Этот метод надо вызывать с объектом класса

In [18]:
a = A()

A.get_value(a)

Object: is A


Эквивалентно

In [19]:
a.get_value()

Object: is A


### Создание классов с помощью `Type`

Динамическое создание простого класса

In [20]:
class A:
    pass

A = type('A', (), {})

Более сложный пример

In [22]:
def f(self): 
    print('method')
    
class A:
    def d(self): 
        print('base method') 
        
B = type('B', (A, ), dict(x=1, f=f)) 

print(type(B))
  
obj = B() 

print(type(obj))
  
obj.d() 
  
obj.f() 
  
print(obj.x) 


<class 'type'>
<class '__main__.B'>
base method
method
1


### Метаклассы

Метаклассы - фабрики по созданию новых классов.

In [23]:
# MyClass = MyMetaClass()
# MyObject = MyClass()

In [24]:
class Meta(type):
    pass

class MyClass(metaclass=Meta):
    pass

class MySubclass(MyClass):
    pass

In [26]:
class MetaClass(type):
    
    def __new__(meta_cls, cls_name, cls_bases, cls_methods):        
        
        new_cls_methods = {}
        for name, val in cls_methods.items():
            if not name.startswith('__'):
                new_cls_methods[name] = val
                new_cls_methods['{}_display'.format(name)] = val
            else:
                new_cls_methods[name] = val
                
        print('__new__ Meta class: {}'.format(meta_cls))
        print('__new__ Class name: {}'.format(cls_name))
        print('__new__ Class bases: {}'.format(cls_bases))
        print('__new__ Class methods: {}'.format(new_cls_methods))                

        return type.__new__(meta_cls, cls_name, cls_bases, new_cls_methods)
    
    def __init__(cls, name, bases, cls_methods):
        
        print('__init__ cls: {}'.format(cls))
        print('__init__ name: {}'.format(name))
        print('__init__ bases: {}'.format(bases))
        print('__init__ methods: {}'.format(bases))            

In [28]:
A = MetaClass('A', (), {'a': 1})
a = A()

__new__ Meta class: <class '__main__.MetaClass'>
__new__ Class name: A
__new__ Class bases: ()
__new__ Class methods: {'a': 1, 'a_display': 1}
__init__ cls: <class '__main__.A'>
__init__ name: A
__init__ bases: ()
__init__ methods: ()


In [29]:
class A (metaclass=MetaClass):
    
    def __init__(self):
        super().__init__()
        
    def f(a):
        print('f method')

__new__ Meta class: <class '__main__.MetaClass'>
__new__ Class name: A
__new__ Class bases: ()
__new__ Class methods: {'__module__': '__main__', '__qualname__': 'A', '__init__': <function A.__init__ at 0x7f14280e0ae8>, 'f': <function A.f at 0x7f14280e0bf8>, 'f_display': <function A.f at 0x7f14280e0bf8>, '__classcell__': <cell at 0x7f1428151be8: empty>}
__init__ cls: <class '__main__.A'>
__init__ name: A
__init__ bases: ()
__init__ methods: ()


Теперь у нас есть два метода вместо одного.

In [31]:
a = A()

a.f()

a.f_display()

f method
f method


### Использование метаклассов для создания базовых абстрактных классов

In [32]:
# источник примера: https://julien.danjou.info/guide-python-static-class-abstract-methods/
from abc import ABCMeta, ABC, abstractmethod

class BasePizza(metaclass=ABCMeta):    

    @staticmethod
    @abstractmethod    
    def get_ingredients():
         """Вернуть список ингридиентов."""

class DietPizza(BasePizza):
    
    @staticmethod
    def get_ingredients():
        return ['рисовая мука', 'салат']

Ошибка. Мы не можем создать объект базового класса.

In [35]:
b = BasePizza()

TypeError: Can't instantiate abstract class BasePizza with abstract methods get_ingredients

Объект производного класса создать уже можем, всё Ок.

In [37]:
d = DietPizza()

d.get_ingredients()

['рисовая мука', 'салат']

### Дополнительно

Больше информации о динамическом создании классов можно найти

https://docs.python.org/3.5/reference/datamodel.html#customizing-class-creation